In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time

import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from multiprocess import Pool

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
Cannot import mxnet.


In [2]:
stack = 'MD642'
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [3]:
setting = 12

In [4]:
if setting == 12:
    available_classifiers = {2: DataManager.load_classifiers(setting=2),
                             10: DataManager.load_classifiers(setting=10)}
else:
    available_classifiers = {setting: DataManager.load_classifiers(setting=setting)}

NameError: global name 'CLF_ROOTDIR' is not defined

In [15]:
def clf_predict(stack, sec):
    
    fn = metadata_cache['sections_to_filenames'][stack][sec]
    if is_invalid(fn):
        return

    try:
        features = DataManager.load_dnn_features(stack=stack, model_name='Sat16ClassFinetuned', section=sec)
    except Exception as e:
        sys.stderr.write('%s\n' % e.message)
        return

    actual_setting = resolve_actual_setting(setting=setting, stack=stack, sec=sec)
    clf_allClasses_ = available_classifiers[actual_setting]

    for structure, clf in clf_allClasses_.iteritems():

        probs = clf.predict_proba(features)[:, clf.classes_.tolist().index(1.)]

        output_fn = DataManager.get_sparse_scores_filepath(stack=stack, structure=structure, 
                                                           setting=actual_setting, sec=sec)
        create_if_not_exists(os.path.dirname(output_fn))

        bp.pack_ndarray_file(probs, output_fn)

In [16]:
clf_predict(stack=stack, sec=203)

In [ ]:
t = time.time()

pool = Pool(8)
pool.map(lambda sec: clf_predict(stack=stack, sec=sec), range(200, 201))
# pool.map(lambda sec: clf_predict(stack=stack, sec=sec), range(first_sec, last_sec+1))
pool.close()
pool.join()

sys.stderr.write('Classifier predict: %.2f\n' % (time.time()-t)) # 30s each section, single process; 